In [1]:
%reload_ext autoreload
%autoreload 2
from alpharaw.ms_data_base import ms_reader_provider
from alpharaw import register_all_readers
from dia_aspire_rescore.io import read_diann2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spectrum_utils.plot as sup
import spectrum_utils.spectrum as sus
from dia_aspire_rescore.psm.matcher import DIAPeptideSpectrumMatcher
from alphabase.peptide.fragment import create_fragment_mz_dataframe, flatten_fragments

register_all_readers()


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pysciexwifffilereader.py:44: UserWarning: Dotnet-based dependencies could not be loaded. Sciex support is disabled.
  warnings.warn(
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pythermorawfilereader.py:35: UserWarning: Dotnet-based dependencies could not be loaded. Thermo support is disabled.
  warnings.warn(


In [2]:
psm_df_original = read_diann2("../../data/raw/SYS026_RA957/DDA_SYSMHC_bynam/lib-base-result-first-pass.parquet")
# decreasing rt order
psm_df_sorted = psm_df_original.sort_values(by="rt", ascending=True)
psm_df_sorted = psm_df_sorted[psm_df_sorted['fdr1_search1'] < 0.01].reset_index(drop=True)
psm_df_sorted

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alphabase/psm_reader/psm_reader.py:342: UserWarning: Unknown modifications: {'Q(UniMod:1)'}. Precursors with unknown modifications will be removed.
  warnings.warn(


,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr1_search1,fdr2_search1,fdr1_search2,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,SSYGEHPKRR,2,0.331321,0.111883,0.488129,0.0,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,,...,3.119479e-03,0.003943,0.0,0.0,,,10,0.002768,608.812710,0.0
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,3,0.382806,0.194710,0.633754,0.0,1/tr|G3V3V3|G3V3V3_HUMAN,1/tr|G3V3V3|G3V3V3_HUMAN,,...,2.237220e-04,1.000000,0.0,0.0,,,10,0.003199,360.208273,0.0
2,20200317_QE_HFX2_LC3_DIA_RA957_R01,SVKNGPGSSR,2,0.388414,0.106270,0.670733,0.0,2/sp|A8MQ11|PM2P5_HUMAN/sp|A4D2B8|PM2P1_HUMAN,2/sp|A8MQ11|PM2P5_HUMAN/sp|A4D2B8|PM2P1_HUMAN,,...,5.522195e-07,0.000124,0.0,0.0,,,10,0.003245,494.762155,0.0
3,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,2,0.390658,0.202562,0.610250,0.0,1/sp|Q86U06|RBM23_HUMAN,1/sp|Q86U06|RBM23_HUMAN,,...,8.692251e-05,0.000124,0.0,0.0,,,10,0.003264,539.808771,0.0
4,20200317_QE_HFX2_LC3_DIA_RA957_R01,HSAHQKYVR,2,0.391779,0.109635,0.674098,0.0,2/sp|P50914|RL14_HUMAN/tr|E7EPB3|E7EPB3_HUMAN,2/sp|P50914|RL14_HUMAN/tr|E7EPB3|E7EPB3_HUMAN,,...,7.780083e-04,0.000835,0.0,0.0,,,9,0.003273,563.299239,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83140,20200317_QE_HFX2_LC3_DIA_RA957_R01,LPMPTEPLL,1,119.558762,119.464920,119.652580,0.0,1/sp|Q9BWH6|RPAP1_HUMAN,1/sp|Q9BWH6|RPAP1_HUMAN,,...,3.904071e-03,0.000124,0.0,0.0,,,9,0.998964,1010.559081,0.0
83141,20200317_QE_HFX2_LC3_DIA_RA957_R02,IPFSGALEL,1,119.560677,119.279823,119.811157,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,8.692251e-05,0.000124,0.0,0.0,,,9,0.999730,946.524410,0.0
83142,20200317_QE_HFX2_LC3_DIA_RA957_R01,YPLPPPVGEQVF,2,119.644463,119.363319,119.863693,0.0,1/sp|Q8TD17|ZN398_HUMAN,1/sp|Q8TD17|ZN398_HUMAN,,...,1.969745e-04,0.000229,0.0,0.0,,,12,0.999681,671.855721,0.0
83143,20200317_QE_HFX2_LC3_DIA_RA957_R01,FPNFTNQLL,2,119.670090,119.576309,119.764122,0.0,1/sp|Q7Z392|TPC11_HUMAN,1/sp|Q7Z392|TPC11_HUMAN,,...,8.692251e-05,0.000124,0.0,0.0,,,9,0.999895,547.287474,0.0


In [3]:
matcher = DIAPeptideSpectrumMatcher(n_neighbors=1)
psm_df, fragment_mz_df, matched_intensity_df, matched_mz_err_df = matcher.match_ms2_multi_raw(
    psm_df_sorted,
    {'20200317_QE_HFX2_LC3_DIA_RA957_R01': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5',
     '20200317_QE_HFX2_LC3_DIA_RA957_R02': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R02.mzML.hdf5'},
    'hdf5',
)

100%|██████████| 2/2 [00:59<00:00, 29.77s/it]


In [4]:
psm_df

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs,frag_start_idx,frag_stop_idx,spec_idx
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,SSYGEHPKRR,2,0.331321,0.111883,0.488129,0.0,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,,...,0.0,,,10,0.002768,608.812710,0.0,0,9,241
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,3,0.382806,0.194710,0.633754,0.0,1/tr|G3V3V3|G3V3V3_HUMAN,1/tr|G3V3V3|G3V3V3_HUMAN,,...,0.0,,,10,0.003199,360.208273,0.0,9,18,277
2,20200317_QE_HFX2_LC3_DIA_RA957_R01,SVKNGPGSSR,2,0.388414,0.106270,0.670733,0.0,2/sp|A8MQ11|PM2P5_HUMAN/sp|A4D2B8|PM2P1_HUMAN,2/sp|A8MQ11|PM2P5_HUMAN/sp|A4D2B8|PM2P1_HUMAN,,...,0.0,,,10,0.003245,494.762155,0.0,18,27,282
3,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,2,0.390658,0.202562,0.610250,0.0,1/sp|Q86U06|RBM23_HUMAN,1/sp|Q86U06|RBM23_HUMAN,,...,0.0,,,10,0.003264,539.808771,0.0,27,36,261
4,20200317_QE_HFX2_LC3_DIA_RA957_R01,HSAHQKYVR,2,0.391779,0.109635,0.674098,0.0,2/sp|P50914|RL14_HUMAN/tr|E7EPB3|E7EPB3_HUMAN,2/sp|P50914|RL14_HUMAN/tr|E7EPB3|E7EPB3_HUMAN,,...,0.0,,,9,0.003273,563.299239,0.0,36,44,262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249430,20200317_QE_HFX2_LC3_DIA_RA957_R02,LPMPTEPLL,1,119.405838,119.250160,119.499657,0.0,1/sp|Q9BWH6|RPAP1_HUMAN,1/sp|Q9BWH6|RPAP1_HUMAN,,...,0.0,,,9,0.998435,1010.559081,0.0,2128334,2128342,88525
249431,20200317_QE_HFX2_LC3_DIA_RA957_R02,LPSIVNEVL,2,119.420029,119.233032,119.544731,0.0,1/sp|Q99623|PHB2_HUMAN,1/sp|Q99623|PHB2_HUMAN,,...,0.0,,,9,0.998554,492.292225,0.0,2128350,2128358,88533
249432,20200317_QE_HFX2_LC3_DIA_RA957_R02,TGIFLPILR,2,119.421150,119.296127,119.545860,0.0,1/tr|C9J8M6|C9J8M6_HUMAN,1/tr|C9J8M6|C9J8M6_HUMAN,,...,0.0,,,9,0.998563,515.326402,0.0,2128358,2128366,88557
249433,20200317_QE_HFX2_LC3_DIA_RA957_R02,FPNFTNQLL,2,119.548103,119.454720,119.610542,0.0,1/sp|Q7Z392|TPC11_HUMAN,1/sp|Q7Z392|TPC11_HUMAN,,...,0.0,,,9,0.999625,547.287474,0.0,2128422,2128430,88651


In [5]:
# find all psm that sequence is 'SSYGEHPKRR' charge is 2
psm_df[(psm_df['sequence'] == 'SSYGEHPKRR') & (psm_df['charge'] == 2)]


,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs,frag_start_idx,frag_stop_idx,spec_idx
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,SSYGEHPKRR,2,0.331321,0.111883,0.488129,0.0,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,,...,0.0,,,10,0.002768,608.81271,0.0,0,9,241
41817,20200317_QE_HFX2_LC3_DIA_RA957_R01,SSYGEHPKRR,2,0.331321,0.111883,0.488129,0.0,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,,...,0.0,,,10,0.002768,608.81271,0.0,709491,709500,264
83634,20200317_QE_HFX2_LC3_DIA_RA957_R01,SSYGEHPKRR,2,0.331321,0.111883,0.488129,0.0,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,3/sp|Q6NYC1|JMJD6_HUMAN/sp|Q6NYC1-3|JMJD6_HUMA...,,...,0.0,,,10,0.002768,608.81271,0.0,1418982,1418991,287


In [6]:
# find all psm that sequence is STIGETSKKK charge is 3
psm_df[(psm_df['sequence'] == 'STIGETSKKK') & (psm_df['charge'] == 3)]

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs,frag_start_idx,frag_stop_idx,spec_idx
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,3,0.382806,0.19471,0.633754,0.0,1/tr|G3V3V3|G3V3V3_HUMAN,1/tr|G3V3V3|G3V3V3_HUMAN,,...,0.0,,,10,0.003199,360.208273,0.0,9,18,277
41818,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,3,0.382806,0.19471,0.633754,0.0,1/tr|G3V3V3|G3V3V3_HUMAN,1/tr|G3V3V3|G3V3V3_HUMAN,,...,0.0,,,10,0.003199,360.208273,0.0,709500,709509,300
83635,20200317_QE_HFX2_LC3_DIA_RA957_R01,STIGETSKKK,3,0.382806,0.19471,0.633754,0.0,1/tr|G3V3V3|G3V3V3_HUMAN,1/tr|G3V3V3|G3V3V3_HUMAN,,...,0.0,,,10,0.003199,360.208273,0.0,1418991,1419000,323


In [7]:
idx = 0
df_r1 = psm_df[psm_df['raw_name'] == '20200317_QE_HFX2_LC3_DIA_RA957_R01'].reset_index(drop=True)
print(matcher.max_spec_per_query)
for i in range(0, matcher.max_spec_per_query):
    current_idx = idx + i * (len(df_r1))//matcher.max_spec_per_query # /2 because of 2 raw files
    print('=======================')
    print(f'idx: {current_idx}')
    print(f'seq: {df_r1.loc[current_idx, 'sequence']}')
    print(f'rt: {df_r1.loc[current_idx, 'rt']}')
    print(f'spec_idx: {df_r1.loc[current_idx, 'spec_idx']}')


3
idx: 0
seq: SSYGEHPKRR
rt: 0.331320583820343
spec_idx: 241
idx: 41817
seq: SSYGEHPKRR
rt: 0.331320583820343
spec_idx: 264
idx: 83634
seq: SSYGEHPKRR
rt: 0.331320583820343
spec_idx: 287


In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def pearson_correlation(a, b):
    return np.corrcoef(a, b)[0, 1]

# test
assert np.isclose(
    cosine_similarity(np.array([1,0]), np.array([0.5, np.sqrt(3)/2])),
    0.5
)
assert pearson_correlation(np.array([1, 2, 3]), np.array([4, 5, 6])) == 1.0

# Test whether the neighbor ms2 in the same isolation window are similar

In [9]:
psm_match = DIAPeptideSpectrumMatcher(n_neighbors=3)
df, fragment_mz_df, inten_df, matched_mz_err_df = psm_match.match_ms2_multi_raw(
    psm_df_sorted[(psm_df_sorted['raw_name'] == '20200317_QE_HFX2_LC3_DIA_RA957_R01') & (psm_df_sorted['fdr1_search1'] < 0.01)],
    {'20200317_QE_HFX2_LC3_DIA_RA957_R01': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5',},
    'hdf5',
)

100%|██████████| 1/1 [01:06<00:00, 66.95s/it]


In [10]:
from peptdeep.model.ms2 import calc_ms2_similarity
# only able to correctly calculate when there is one raw file
metrics_list = []
frag_len = len(inten_df)//psm_match.max_spec_per_query
_df = df.iloc[:len(df)//psm_match.max_spec_per_query].copy()
for i in range(psm_match.max_spec_per_query):
    for j in range(i+1,psm_match.max_spec_per_query):
        _df,metrics_df = calc_ms2_similarity(
            _df,
            inten_df[i*frag_len:(i+1)*frag_len],
            inten_df[j*frag_len:(j+1)*frag_len],
        )
    metrics_list.append(metrics_df)
sum(metrics_list)/len(metrics_list)

,PCC,COS,SA,SPC
count,41817.000000,41817.000000,41817.000000,41817.000000
mean,0.653760,0.682468,0.542366,0.533019
std,0.311449,0.291915,0.269205,0.264473
min,-0.156912,0.000000,0.000000,-0.667367
25%,0.460212,0.513676,0.353770,0.360664
50%,0.743622,0.769774,0.574593,0.553888
75%,0.910446,0.918157,0.753533,0.734442
max,1.000000,1.000000,1.000000,1.000000
>0.90,0.324263,0.342058,0.096048,0.064280
>0.75,0.524804,0.551467,0.291655,0.229893
